In [1]:
import os
import uproot4
import awkward1 as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep
import numba as nb
import h5py

In [2]:
from create_table import create_table

In [3]:
help( ak.highlevel.Array )

Help on class Array in module awkward1.highlevel:

class Array(numpy.lib.mixins.NDArrayOperatorsMixin, awkward1._connect._pandas.PandasMixin)
 |  Array(data, behavior=None, with_name=None, check_valid=False)
 |  
 |  Args:
 |      data (#ak.layout.Content, #ak.Array, np.ndarray, str, or iterable):
 |          Data to wrap or convert into an array.
 |          If a NumPy array, the regularity of its dimensions is preserved
 |          and the data are viewed, not copied.
 |          If a string, the data are assumed to be JSON.
 |          If an iterable, calls #ak.from_iter, which assumes all dimensions
 |          have irregular lengths.
 |      behavior (None or dict): Custom #ak.behavior for this Array only.
 |      with_name (None or str): Gives tuples and records a name that can be
 |          used to override their behavior (see below).
 |      check_valid (bool): If True, verify that the #layout is valid.
 |  
 |  High-level array that can contain data of any type.
 |  
 |  For 

In [4]:
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-Dilepton/190621_214154/")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0001")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W2e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160449/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPSp,.nGGToWW_bSM-A0W5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160355/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWWToJJMuNu_PtL-15_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160255/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWWToJJENu_PtL-15_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160227/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185019/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW8e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185049/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW2e-5_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185121/0000")

In [5]:
#fileName_ = "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root"
fileName_ = "/eos/home-a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root"
#fileName_ = "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000/SlimmedNtuple_merged_0.root"
root_ = uproot4.open( fileName_ )
print ( "Number of events in tree: {}".format( np.array( root_["demo/SlimmedNtuple/event"] ).size ) )

tree_ = root_["demo/SlimmedNtuple"]
 
keys_nonproton = ["run", "event", "lumiblock", "nVertices",
                  "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
                  "pfcand_nextracks", "pfcand_nextracks_noDRl",
                  "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi"]
keys_nonproton.extend( tree_.keys( filter_name="jet*") )
keys_nonproton.extend( tree_.keys( filter_name="muon*") )
keys_nonproton.extend( tree_.keys( filter_name="met*") )
keys_proton = tree_.keys( filter_name="proton*")
keys_ppstrack = tree_.keys( filter_name="pps_track*")
keys = []
keys.extend( keys_nonproton )
keys.extend( keys_proton )
keys.extend( keys_ppstrack )
print ( keys )

how_ = None
#how_= "zip"

keys_extra = [ 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift2', 'proton_rpid2' ]
for key_ in keys_extra:
    if key_ in keys: keys.remove( key_ )
print ( keys )

arrays = tree_.arrays( keys, library="ak", how=how_, entry_stop=10 )
print ( "\n" )
print ( arrays )
print ( ak.to_list( arrays ) )

arrays_extra = tree_.arrays( keys_extra, library="ak", how=how_, entry_stop=10 )
print ( "\n" )
print ( arrays_extra )
print ( ak.to_list( arrays_extra ) )


Number of events in tree: 74007
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift1', 'proton_trackpixshift2', 'proton_rpid1', 'proton_rpid2', 'pps_track_x', 'pps_track_y', 'pps_track_rpid']
['run', 'event', 'lumiblock', 'nVertices', 

In [6]:
if how_ == "zip":
    protons = arrays["proton"]
    protons["run"] = arrays["run"]
    protons["event"] = arrays["event"]
    protons["lumiblock"] = arrays["lumiblock"]
    print ( protons )
    print ( ak.to_list(protons) )
    
    ppstracks = arrays["pps_track"]
    ppstracks["run"] = arrays["run"]
    ppstracks["event"] = arrays["event"]
    ppstracks["lumiblock"] = arrays["lumiblock"]
    print ("\n")
    print ( ppstracks )
    print ( ak.to_list(ppstracks) )
    
    protons_extra = arrays_extra[ "proton" ]
    protons_multiRP = protons[ protons.ismultirp == 1 ]
    protons_multiRP[ "trackx2" ] = protons_extra[ "trackx2" ]
    protons_multiRP[ "tracky2" ] = protons_extra[ "tracky2" ]
    protons_multiRP[ "trackpixshift2" ] = protons_extra[ "trackpixshift2" ]
    protons_multiRP[ "rpid2" ] = protons_extra[ "rpid2" ]
    print ("\n")
    print ( protons_multiRP )
    print ( ak.to_list( protons_multiRP ) )  
    
else:
    arrays_proton = {}
    
    keys_ = keys_proton.copy()
    for key_ in keys_extra:
        if key_ in keys_: keys_.remove( key_ )
    print ( keys_ )
    
    for key_ in keys_: arrays_proton[ key_[ len("proton_") : ] ] = arrays[ key_ ]
    print ( arrays_proton )
    protons = ak.zip( arrays_proton )
    print ( protons )
    print ( ak.to_list( protons ) )
    
    arrays_proton_extra = {}
    
    for key_ in keys_extra: arrays_proton_extra[ key_[ len("proton_") : ] ] = arrays_extra[ key_ ]
    print ("\n")
    print ( arrays_proton_extra )
    protons_extra = ak.zip( arrays_proton_extra )
    print ( protons_extra )
    print ( ak.to_list( protons_extra ) )
    
    protons_multiRP = protons[ protons.ismultirp_ == 1 ]
    protons_multiRP[ "trackx2" ] = protons_extra[ "trackx2" ]
    protons_multiRP[ "tracky2" ] = protons_extra[ "tracky2" ]
    protons_multiRP[ "trackpixshift2" ] = protons_extra[ "trackpixshift2" ]
    protons_multiRP[ "rpid2" ] = protons_extra[ "rpid2" ]
    print ("\n")
    print ( protons_multiRP )
    print ( ak.to_list( protons_multiRP ) )  
        

['proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', 'proton_trackpixshift1', 'proton_rpid1']
{'xi': <Array [[0.122], [0.0467, ... [], [0.133]] type='10 * var * float32'>, 'thx': <Array [[0], [0, 0, 0, 0, 0, ... [0], [], [0]] type='10 * var * float32'>, 'thy': <Array [[-0.00014], ... [-0.000134]] type='10 * var * float32'>, 't': <Array [[-0.742], [-0.46, ... [], [-0.676]] type='10 * var * float32'>, 'ismultirp_': <Array [[0], [0, 0, 0, 0, 0, ... [0], [], [0]] type='10 * var * int32'>, 'rpid': <Array [[3], [3, 23, 23, ... 103], [], [103]] type='10 * var * int32'>, 'arm': <Array [[-999], [-999, -999, ... [], [-999]] type='10 * var * int32'>, 'time': <Array [[0], [0, 0, 0, 0, 0, ... [0], [], [0]] type='10 * var * float32'>, 'trackx1': <Array [[13.2], [4.43, 4.29, ... [], [9.66]] type='10 * var * float32'>, 'tracky1': <Array [[7.49], [1.13, 0.473, ... [], [8.13]] type='10 * var * float32'>

In [7]:
# def create_table( fileNames, label, random_protons=False, resample_factor=-1, read_size="150MB", firstEvent=None, entryStop=None, debug=False ):

#     fileNames_ = fileNames
#     label_ = label
#     random_protons_ = random_protons
#     resample_factor_ = resample_factor
#     read_size_ = read_size
#     firstEvent_ = firstEvent
#     entryStop_ = entryStop

#     #how_ = None
#     how_ = "zip"
    
#     print ( "Random protons: {}".format( random_protons_ ) )
    
#     resample = False
#     if resample_factor_ > 1: resample = True
#     print ( "Resample: {} / Resample factor: {}".format( resample,  resample_factor_ ) )

#     np.random.seed( 42 )

#     dset_chunk_size = 50000

#     columns_protons = ( "run", "lumiblock", "event", "slice", "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm",
#                         "jet0_pt", "jet0_eta", "jet0_phi", "jet0_energy", "jet0_mass", "jet0_corrmass", "jet0_tau1", "jet0_tau2", "jet0_vertexz",
#                         "muon0_pt", "muon0_eta", "muon0_phi", "muon0_energy", "muon0_charge", "muon0_iso", "muon0_dxy", "muon0_dz",
#                         "met", "met_x", "met_y", "met_phi",
#                         "nVertices",
#                         "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
#                         "pfcand_nextracks", "pfcand_nextracks_noDRl",
#                         "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi" )

#     columns_ppstracks = ( "run", "lumiblock", "event", "slice", "x", "y", "rpid" ) 

#     protons_keys = {}
#     for col_ in columns_protons:
#         protons_keys[ col_ ] = col_

#     ppstracks_keys = {}
#     for col_ in columns_ppstracks:
#         ppstracks_keys[ col_ ] = col_

#     counts_label_protons_ = "Proton" if not random_protons_ else "ProtonRnd"

#     with h5py.File( 'output-' + label_ + '.h5', 'w') as f:

#         dset_protons_multiRP = f.create_dataset( 'protons_multiRP', ( dset_chunk_size, len( columns_protons ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_protons ) ) )
#         print ( "Initial dataset shape: {}".format( dset_protons_multiRP.shape ) )

#         dset_protons_singleRP = f.create_dataset( 'protons_singleRP', ( dset_chunk_size, len( columns_protons ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_protons ) ) )
#         print ( "Initial dataset shape: {}".format( dset_protons_singleRP.shape ) )

#         dset_ppstracks = f.create_dataset( 'ppstracks', ( dset_chunk_size, len( columns_ppstracks ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_ppstracks ) ) )
#         print ( "Initial dataset shape: {}".format( dset_ppstracks.shape ) )

#         protons_multiRP_list = {}
#         for col_ in columns_protons:
#             protons_multiRP_list[ col_ ] = []           

#         protons_singleRP_list = {}
#         for col_ in columns_protons:
#             protons_singleRP_list[ col_ ] = []           

#         ppstracks_list = {}
#         for col_ in columns_ppstracks:
#             ppstracks_list[ col_ ] = []           

#         selections = None
#         counts = None

#         dset_multiRP_slice = 0
#         dset_multiRP_idx = 0
#         dset_multiRP_entries = 0

#         dset_singleRP_slice = 0
#         dset_singleRP_idx = 0
#         dset_singleRP_entries = 0

#         dset_ppstracks_slice = 0
#         dset_ppstracks_idx = 0
#         dset_ppstracks_entries = 0

#         for file_ in fileNames_:
#             print ( file_ ) 
#             root_ = uproot4.open( file_ )

#             print ( "Number of events in tree: {}".format( np.array( root_["demo/SlimmedNtuple/event"] ).size ) )

#             tree_ = root_["demo/SlimmedNtuple"]

#             keys_nonproton = [ "run", "event", "lumiblock", "nVertices",
#                                "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
#                                "pfcand_nextracks", "pfcand_nextracks_noDRl",
#                                "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi" ]
#             keys_nonproton.extend( tree_.keys( filter_name="jet*") )
#             keys_nonproton.extend( tree_.keys( filter_name="muon*") )
#             keys_nonproton.extend( tree_.keys( filter_name="met*") )
#             keys = []
#             keys.extend( keys_nonproton )
#             keys.extend( tree_.keys( filter_name="proton*") )
#             keys.extend( tree_.keys( filter_name="pps*") )
#             keys_remove = [ 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift2', 'proton_rpid2' ]
#             for key_ in keys_remove:
#                 if key_ in keys: keys.remove( key_ )
#             print ( keys )

#             for events_ in tree_.iterate( keys , library="ak", how=how_, step_size=read_size_, entry_start=firstEvent_, entry_stop=entryStop_ ):
#                 print ( len(events_), events_ )
#                 print ( "Num jets: {}".format( ak.num( events_["jet"] ) ) )
#                 print ( "Num muons: {}".format( ak.num( events_["muon"] ) ) )
#                 print ( "Num protons: {}".format( ak.num( events_["proton"] ) ) )
#                 print ( "Num pps tracks: {}".format( ak.num( events_["pps_track"] ) ) )

#                 selections_ = []
#                 counts_ = []

#                 selections_.append( "All" )
#                 counts_.append( len( events_ ) )

#                 # Event selections
#                 msk_1jet = ( ak.num( events_["jet"] ) >= 1 )
#                 selections_.append( "Jet" )
#                 counts_.append( np.sum( np.array( msk_1jet ).astype("int32") ) )

#                 msk_1muon = msk_1jet & ( ak.num( events_["muon"] ) >= 1 )
#                 selections_.append( "Muon" )
#                 counts_.append( np.sum( np.array( msk_1muon ).astype("int32") ) )

#                 events_ = events_[ msk_1muon ]    

#                 selections_ = np.array( selections_ )
#                 counts_ = np.array( counts_ )

#                 # Repeat events by resample factor
#                 if resample:
#                     counts_ = counts_ * resample_factor_

#                 if selections is None:
#                     selections = selections_
#                     counts = counts_
#                 else:
#                     msk_selections = np.full_like( selections, False, dtype='bool' )
#                     for key in selections_:
#                         msk_selections |= ( selections == key )
#                     counts[ msk_selections ] += counts_

#                 # Repeat events by resample factor
#                 slices_ = np.zeros( len( events_ ), dtype=np.int32 )
#                 if resample:
#                     events_size_ = len( events_ )
#                     events_ = ak.concatenate( ( [events_] * resample_factor_ ), axis=0 )
#                     slices_ = np.zeros( resample_factor_ * events_size_, dtype=np.int32 )
#                     for idx_ in range( resample_factor_ ):
#                         slices_[ ( idx_ * events_size_ ) : ( ( idx_ + 1 ) * events_size_ ) ] = idx_

#                 events_[ "slice" ] = slices_

#                 # Randomize proton arrays
#                 #protons_ = events_["proton"]
#                 protons_ = None
#                 ppstracks_ = None
#                 if random_protons_:
#                     protons_ = events_["proton"]
#                     ppstracks_ = events_["pps_track"]

#                     index_rnd_ = np.random.permutation( len( events_ ) )

#                     protons_rnd_ = protons_[ index_rnd_ ]
#                     ppstracks_rnd_ = ppstracks_[ index_rnd_ ]

#                     events_[ "proton_rnd" ] = protons_rnd_
#                     events_[ "pps_track_rnd" ] = ppstracks_rnd_

#                     print ( "Num protons: {}".format( ak.num( events_["proton"] ) ) )
#                     print ( "Num protons randomized: {}".format( ak.num( events_["proton_rnd"] ) ) )
#                     print ( "Num pps tracks: {}".format( ak.num( events_["pps_track"] ) ) )
#                     print ( "Num pps tracks randomized: {}".format( ak.num( events_["pps_track_rnd"] ) ) )

#                 if not random_protons_:
#                     protons_ = events_["proton"]
#                     ppstracks_ = events_["pps_track"]
#                 else:
#                     protons_ = events_["proton_rnd"]
#                     ppstracks_ = events_["pps_track_rnd"]
                    
#                 print ( "Num protons: {}".format( ak.num( protons_ ) ) )
#                 print ( "Num pps tracks: {}".format( ak.num( ppstracks_ ) ) )

#                 protons_["run"]                    = events_["run"]
#                 protons_["lumiblock"]              = events_["lumiblock"]
#                 protons_["event"]                  = events_["event"]
#                 protons_["slice"]                  = events_["slice"]
#                 protons_["jet0_pt"]                = events_.jet.pt[:,0]
#                 protons_["jet0_eta"]               = events_.jet.eta[:,0]
#                 protons_["jet0_phi"]               = events_.jet.phi[:,0]
#                 protons_["jet0_energy"]            = events_.jet.energy[:,0]
#                 protons_["jet0_mass"]              = events_.jet.mass[:,0]
#                 protons_["jet0_corrmass"]          = events_.jet.corrmass[:,0]
#                 protons_["jet0_tau1"]              = events_.jet.tau1[:,0]
#                 protons_["jet0_tau2"]              = events_.jet.tau2[:,0]
#                 protons_["jet0_vertexz"]           = events_.jet.vertexz[:,0]
#                 protons_["muon0_pt"]               = events_.muon.pt[:,0]
#                 protons_["muon0_eta"]              = events_.muon.eta[:,0]
#                 protons_["muon0_phi"]              = events_.muon.phi[:,0]
#                 protons_["muon0_energy"]           = events_.muon.e[:,0]
#                 protons_["muon0_charge"]           = events_.muon.charge[:,0]
#                 protons_["muon0_iso"]              = events_.muon.iso[:,0]
#                 protons_["muon0_dxy"]              = events_.muon.dxy[:,0]
#                 protons_["muon0_dz"]               = events_.muon.dz[:,0]
#                 protons_["met"]                    = events_["met"]
#                 protons_["met_x"]                  = events_["met_x"]
#                 protons_["met_y"]                  = events_["met_y"]
#                 protons_["met_phi"]                = events_["met_phi"]
#                 protons_["nVertices"]              = events_["nVertices"]
#                 protons_["num_bjets_ak8"]          = events_["num_bjets_ak8"]
#                 protons_["num_bjets_ak4"]          = events_["num_bjets_ak4"]
#                 protons_["num_jets_ak4"]           = events_["num_jets_ak4"]
#                 protons_["pfcand_nextracks"]       = events_["pfcand_nextracks"]
#                 protons_["pfcand_nextracks_noDRl"] = events_["pfcand_nextracks_noDRl"]
#                 protons_["recoMWhad"]              = events_["recoMWhad"]
#                 protons_["recoMWlep"]              = events_["recoMWlep"]
#                 protons_["recoMWW"]                = events_["recoMWW"]
#                 protons_["recoRapidityWW"]         = events_["recoRapidityWW"]
#                 protons_["dphiWW"]                 = events_["dphiWW"]
#                 protons_["WLeptonicPt"]            = events_["WLeptonicPt"]
#                 protons_["WLeptonicPhi"]           = events_["WLeptonicPhi"]
#                 #protons_["x1"] = -999.
#                 #protons_["y1"] = -999.
#                 #protons_["x2"] = -999.
#                 #protons_["y2"] = -999.
                
#                 #ppstracks_ = events_["pps_track"]
#                 ppstracks_["run"] = events_["run"]
#                 ppstracks_["lumiblock"] = events_["lumiblock"]
#                 ppstracks_["event"] = events_["event"]
#                 ppstracks_["slice"] = events_["slice"]

#                 protons_singleRP_ = protons_[ protons_.ismultirp == 0 ]
#                 protons_multiRP_ = protons_[ protons_.ismultirp == 1 ]

#                 protons_singleRP_byRP_ = {}
#                 ppstracks_byRP_ = {}
#                 protons_multiRP_byArm_ = {}
#                 for rpid in ( 3, 23, 103, 123 ):
#                     #arm = -1
#                     #if   rpid == 3   or rpid == 23 : arm = 0
#                     #elif rpid == 103 or rpid == 123 : arm = 1
#                     #print ( "Arm: {}".format( arm ) )

#                     protons_singleRP_byRP_[ rpid ] =  protons_singleRP_[ protons_singleRP_.rpid == rpid ]
#                     ppstracks_byRP_[ rpid ] = ppstracks_[ ppstracks_.rpid == rpid ]
#                     #protons_singleRP_byRP_[ rpid ]["x1"] = ppstracks_byRP_[ rpid ].x
#                     #protons_singleRP_byRP_[ rpid ]["y1"] = ppstracks_byRP_[ rpid ].y

#                     print ( "\nNum protons RP {}: {}".format( rpid, ak.num( protons_singleRP_byRP_[ rpid ] ) ) )
#                     if debug:
#                         print ( ak.to_list( protons_singleRP_byRP_[ rpid ] ) )
#                         print ("\n")
#                         print ( ak.to_list( ppstracks_byRP_[ rpid ] ) )

#                 for arm in ( 0, 1 ):
#                     protons_multiRP_byArm_[ arm ] = protons_multiRP_[ protons_multiRP_.arm == arm ]

#                     print ( "\nNum multi-RP protons Arm {}: {}".format( arm, ak.num( protons_multiRP_byArm_[ arm ] ) ) )
#                     if debug:
#                         print ( ak.to_list( protons_multiRP_byArm_[ arm ] ) )

#                 #msk  =  np.array( ak.num( protons_singleRP_byRP_[ 3 ].xi ) == 1 )
#                 #msk &= np.array( ak.num( protons_singleRP_byRP_[ 23 ].xi ) == 1 )
#                 #msk &= np.array( ak.num( protons_singleRP_byRP_[ 103 ].xi ) == 1 )
#                 #msk &= np.array( ak.num( protons_singleRP_byRP_[ 123 ].xi ) == 1 )    

#                 msk_protons  = np.array( ak.num( protons_multiRP_byArm_[ 0 ] ) > 0 )
#                 msk_protons &= np.array( ak.num( protons_multiRP_byArm_[ 1 ] ) > 0 )

#                 protons_multiRP_sel_ = protons_multiRP_[ msk_protons ]
#                 protons_singleRP_sel_ = protons_singleRP_[ msk_protons ]
#                 ppstracks_sel_ = ppstracks_[ msk_protons ]
#                 print ("\n")
#                 if debug:
#                     print ( msk_protons )
#                 print ( len( protons_multiRP_sel_ ) )
#                 print ( ak.num( protons_multiRP_sel_ ) )
#                 if debug:
#                     print ("\n")
#                     print ( ak.to_list( protons_multiRP_sel_ ) )
#                     print ("\n")
#                     print ( ak.to_list( protons_singleRP_sel_ ) )
#                     print ("\n")
#                     print ( ak.to_list( ppstracks_sel_ ) )

#                 counts_protons_ = len( protons_[ msk_protons ] )
#                 if not counts_label_protons_ in selections:
#                     selections = np.concatenate( ( selections, np.array( [ counts_label_protons_ ] ) ) )
#                     counts = np.concatenate( ( counts, np.array( [counts_protons_] ) ) )
#                 else:    
#                     counts[ selections == counts_label_protons_ ] += counts_protons_ 

#                 print ( selections )
#                 print ( counts )

#                 for col_ in columns_protons:
#                     protons_multiRP_list[ col_ ] = np.array( ak.flatten( protons_multiRP_sel_[ protons_keys[ col_ ] ] ) )

#                 arr_size_multiRP_ = len( protons_multiRP_list[ "xi" ] )
#                 print ( "Flattened array size multi-RP: {}".format( arr_size_multiRP_ ) )

#                 for col_ in columns_protons:
#                     protons_singleRP_list[ col_ ] = np.array( ak.flatten( protons_singleRP_sel_[ protons_keys[ col_ ] ] ) )

#                 arr_size_singleRP_ = len( protons_singleRP_list[ "xi" ] )
#                 print ( "Flattened array size single-RP: {}".format( arr_size_singleRP_ ) )

#                 for col_ in columns_ppstracks:
#                     ppstracks_list[ col_ ] = np.array( ak.flatten( ppstracks_sel_[ ppstracks_keys[ col_ ] ] ) )

#                 arr_size_ppstracks_ = len( ppstracks_list[ "x" ] )
#                 print ( "Flattened array size tracks: {}".format( arr_size_ppstracks_ ) )

#                 dset_multiRP_entries += arr_size_multiRP_
#                 dset_singleRP_entries += arr_size_singleRP_
#                 dset_ppstracks_entries += arr_size_ppstracks_

#                 if dset_multiRP_entries > dset_chunk_size:
#                     resize_factor_ = ( dset_multiRP_entries // dset_chunk_size )
#                     chunk_resize_  = resize_factor_ * dset_chunk_size

#                     print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
#                     dset_protons_multiRP.resize( ( dset_protons_multiRP.shape[0] + chunk_resize_ ), axis=0 )
#                     print ( "Dataset shape: {}".format( dset_protons_multiRP.shape ) )

#                     dset_multiRP_slice += resize_factor_
#                     # Count the rest to the chunk size 
#                     dset_multiRP_entries = ( dset_multiRP_entries % dset_chunk_size )

#                 if dset_singleRP_entries > dset_chunk_size:
#                     resize_factor_ = ( dset_singleRP_entries // dset_chunk_size )
#                     chunk_resize_  = resize_factor_ * dset_chunk_size

#                     print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
#                     dset_protons_singleRP.resize( ( dset_protons_singleRP.shape[0] + chunk_resize_ ), axis=0 )
#                     print ( "Dataset shape: {}".format( dset_protons_singleRP.shape ) )

#                     dset_singleRP_slice += resize_factor_
#                     # Count the rest to the chunk size 
#                     dset_singleRP_entries = ( dset_singleRP_entries % dset_chunk_size )

#                 if dset_ppstracks_entries > dset_chunk_size:
#                     resize_factor_ = ( dset_ppstracks_entries // dset_chunk_size )
#                     chunk_resize_  = resize_factor_ * dset_chunk_size

#                     print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
#                     dset_ppstracks.resize( ( dset_ppstracks.shape[0] + chunk_resize_ ), axis=0 )
#                     print ( "Dataset shape: {}".format( dset_ppstracks.shape ) )

#                     dset_ppstracks_slice += resize_factor_
#                     # Count the rest to the chunk size 
#                     dset_ppstracks_entries = ( dset_ppstracks_entries % dset_chunk_size )

#                 print ( "Stacking data." )
#                 data_protons_multiRP_ = np.stack( list( protons_multiRP_list.values() ), axis=1 )
#                 print ( data_protons_multiRP_.shape )
#                 print ( data_protons_multiRP_ )

#                 data_protons_singleRP_ = np.stack( list( protons_singleRP_list.values() ), axis=1 )
#                 print ( data_protons_singleRP_.shape )
#                 print ( data_protons_singleRP_ )

#                 data_ppstracks_ = np.stack( list( ppstracks_list.values() ), axis=1 )
#                 print ( data_ppstracks_.shape )
#                 print ( data_ppstracks_ )

#                 dset_idx_next_ = dset_multiRP_idx + arr_size_multiRP_
#                 print ( "Slice: {}".format( dset_multiRP_slice ) )
#                 print ( "Writing in positions ({},{})".format( dset_multiRP_idx, dset_idx_next_ ) )
#                 dset_protons_multiRP[ dset_multiRP_idx : dset_idx_next_ ] = data_protons_multiRP_
#                 dset_multiRP_idx = dset_idx_next_ 

#                 dset_idx_next_ = dset_singleRP_idx + arr_size_singleRP_
#                 print ( "Slice: {}".format( dset_singleRP_slice ) )
#                 print ( "Writing in positions ({},{})".format( dset_singleRP_idx, dset_idx_next_ ) )
#                 dset_protons_singleRP[ dset_singleRP_idx : dset_idx_next_ ] = data_protons_singleRP_
#                 dset_singleRP_idx = dset_idx_next_ 

#                 dset_idx_next_ = dset_ppstracks_idx + arr_size_ppstracks_
#                 print ( "Slice: {}".format( dset_ppstracks_slice ) )
#                 print ( "Writing in positions ({},{})".format( dset_ppstracks_idx, dset_idx_next_ ) )
#                 dset_ppstracks[ dset_ppstracks_idx : dset_idx_next_ ] = data_ppstracks_
#                 dset_ppstracks_idx = dset_idx_next_ 

#             # Iteration on input files
#             root_.close()

#         # Reduce dataset to its final size 
#         print ( "Reduce dataset." )
#         dset_protons_multiRP.resize( ( dset_multiRP_idx ), axis=0 ) 
#         print ( "Dataset shape: {}".format( dset_protons_multiRP.shape ) )

#         dset_protons_singleRP.resize( ( dset_singleRP_idx ), axis=0 ) 
#         print ( "Dataset shape: {}".format( dset_protons_singleRP.shape ) )

#         dset_ppstracks.resize( ( dset_ppstracks_idx ), axis=0 ) 
#         print ( "Dataset shape: {}".format( dset_ppstracks.shape ) )

#         print ( "Writing column names and event counts.")

#         columns_protons_ = np.array( columns_protons, dtype='S' )
#         print ( columns_protons_ )

#         columns_ppstracks_ = np.array( columns_ppstracks, dtype='S' )
#         print ( columns_ppstracks_ )

#         event_counts_ = counts
#         print ( event_counts_ )

#         selections_ = np.array( selections, dtype='S' )
#         print ( selections_ )

#         dset_columns_protons = f.create_dataset( 'columns_protons', data=columns_protons_ )
#         dset_columns_ppstracks = f.create_dataset( 'columns_ppstracks', data=columns_ppstracks_ )
#         dset_counts = f.create_dataset( 'event_counts', data=event_counts_ )
#         dset_selections = f.create_dataset( 'selections', data=selections_ )

#         print ( dset_protons_multiRP )
#         print ( dset_protons_multiRP[-1] )
#         print ( dset_protons_singleRP )
#         print ( dset_protons_singleRP[-1] )   
#         print ( dset_ppstracks )
#         print ( dset_ppstracks[-1] )   

#         print ( dset_columns_protons )
#         print ( list( dset_columns_protons ) )
#         print ( dset_columns_ppstracks )
#         print ( list( dset_columns_ppstracks ) )   
#         print ( dset_counts )
#         print ( list( dset_counts ) )
#         print ( dset_selections )
#         print ( list( dset_selections ) )

In [8]:
run_tables = True

debug = False

fileNames_A0W_ACW = {}
fileNames_A0W_ACW[ "A0W1e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "A0W2e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W2e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160449/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "A0W5e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160355/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "ACW5e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW5e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185019/0000/SlimmedNtuple_1.root"
]
fileNames_A0W_ACW[ "ACW8e-6" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW8e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185049/0000/SlimmedNtuple_merged_0.root"
]
fileNames_A0W_ACW[ "ACW2e-5" ] = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW2e-5_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185121/0000/SlimmedNtuple_merged_0.root"
]

firstEvent = None
entryStop = None
#read_size = "150MB"
step_size = 100000

if run_tables:
    for key_ in fileNames_A0W_ACW:
        print ( key_, fileNames_A0W_ACW[ key_ ] )
        label = "GGToWW-AQGC-test-" + key_
        create_table( fileNames_A0W_ACW[ key_ ], label=label, random_protons=False, resample_factor=-1, step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        

A0W1e-6 ['/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000/SlimmedNtuple_merged_0.root']
Random protons: False
Resample: False / Resample factor: -1
Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W1e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160422/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 34159
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'm

A0W2e-6 ['/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W2e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160449/0000/SlimmedNtuple_merged_0.root']
Random protons: False
Resample: False / Resample factor: -1
Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-A0W2e-6_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200420_160449/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 74109
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'm

Number of events in tree: 190187
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift1', 'proton_trackpixshift2', 'proton_rpid1', 'proton_rpid2', 'pps_track_x', 'pps_track_y', 'pps_track_rpid']
100000 [{run: 1, event: 104807, lumiblock:

Slice: 1
Writing in positions (36088,69092)
Reduce dataset.
Dataset shape: (34546, 53)
Dataset shape: (69092, 49)
Dataset shape: (69092, 7)
Writing column names and event counts.
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0_mass' b'jet0_corrmass' b'jet0_tau1' b'jet0_tau2'
 b'jet0_vertexz' b'muon0_pt' b'muon0_eta' b'muon0_phi' b'muon0_energy'
 b'muon0_charge' b'muon0_iso' b'muon0_dxy' b'muon0_dz' b'met' b'met_x'
 b'met_y' b'met_phi' b'nVertices' b'num_bjets_ak8' b'num_bjets_ak4'
 b'num_jets_ak4' b'pfcand_nextracks' b'pfcand_nextracks_noDRl'
 b'recoMWhad' b'recoMWlep' b'recoMWW' b'recoRapidityWW' b'dphiWW'
 b'WLeptonicPt' b'WLeptonicPhi' b'trackx1' b'tracky1' b'trackpixshift1'
 b'rpid1' b'trackx2' b'tracky2' b'trackpixshift2' b'rpid2']
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0

Flattened array size multi-RP: 1172
Flattened array size single-RP: 2344
Flattened array size tracks: 2344
Stacking data.
(1172, 53)
[[ 1.00000000e+00  2.54000000e+02  2.53170000e+04 ...  4.99969482e-01
   0.00000000e+00  2.30000000e+01]
 [ 1.00000000e+00  2.54000000e+02  2.53170000e+04 ... -1.40002441e+00
   0.00000000e+00  1.23000000e+02]
 [ 1.00000000e+00  2.54000000e+02  2.53460000e+04 ...  1.59997559e+00
   0.00000000e+00  2.30000000e+01]
 ...
 [ 1.00000000e+00  6.65000000e+03  1.64932000e+05 ...  7.99987793e-01
   0.00000000e+00  1.23000000e+02]
 [ 1.00000000e+00  6.65100000e+03  1.65031000e+05 ...  4.25000000e+00
   0.00000000e+00  2.30000000e+01]
 [ 1.00000000e+00  6.65100000e+03  1.65031000e+05 ...  2.69995117e+00
   0.00000000e+00  1.23000000e+02]]
(2344, 49)
[[ 1.00000000e+00  2.54000000e+02  2.53170000e+04 ...  4.80255127e-01
   0.00000000e+00  3.00000000e+00]
 [ 1.00000000e+00  2.54000000e+02  2.53170000e+04 ...  4.99969482e-01
   0.00000000e+00  2.30000000e+01]
 [ 1.00000

91967 [{run: 1, event: 80915, lumiblock: 810, nVertices: 31, ... pps_track_rpid: []}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, ... 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0]
Num protons: [6, 0, 1, 3, 3, 3, 3, 6, 1, 0, 3, 4, 6, 3, ... 3, 3, 0, 3, 6, 3, 3, 0, 0, 3, 6, 1, 0]
Num pps tracks: [4, 0, 1, 2, 2, 2, 2, 4, 1, 0, 2, 3, 4, 2, ... 2, 2, 0, 2, 4, 2, 2, 0, 0, 2, 4, 1, 0]
Num protons: [6, 0, 3, 3, 3, 6, 1, 4, 3, 3, 3, 3, 1, 0, ... 3, 3, 3, 6, 4, 3, 3, 6, 3, 3, 0, 3, 1]
Num pps tracks: [4, 0, 2, 2, 2, 4, 1, 3, 2, 2, 2, 2, 1, 0, ... 2, 2, 2, 4, 3, 2, 2, 4, 2, 2, 0, 2, 1]
Num protons extra: [2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 0, ... 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 0]

Num protons RP 3: [1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, ... 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]

Num protons RP 23: [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, ... 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]

Nu

Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/GGToWW_bSM-ACW2e-5_13TeV-fpmc-herwig6-signal-proton-propagation-dilepton/200424_185121/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 197331
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 

Slice: 1
Writing in positions (37016,73096)
Reduce dataset.
Dataset shape: (36548, 53)
Dataset shape: (73096, 49)
Dataset shape: (73096, 7)
Writing column names and event counts.
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0_mass' b'jet0_corrmass' b'jet0_tau1' b'jet0_tau2'
 b'jet0_vertexz' b'muon0_pt' b'muon0_eta' b'muon0_phi' b'muon0_energy'
 b'muon0_charge' b'muon0_iso' b'muon0_dxy' b'muon0_dz' b'met' b'met_x'
 b'met_y' b'met_phi' b'nVertices' b'num_bjets_ak8' b'num_bjets_ak4'
 b'num_jets_ak4' b'pfcand_nextracks' b'pfcand_nextracks_noDRl'
 b'recoMWhad' b'recoMWlep' b'recoMWW' b'recoRapidityWW' b'dphiWW'
 b'WLeptonicPt' b'WLeptonicPhi' b'trackx1' b'tracky1' b'trackpixshift1'
 b'rpid1' b'trackx2' b'tracky2' b'trackpixshift2' b'rpid2']
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0

In [10]:
run_tables = True

debug = False

fileNames_data = {}
fileNames_data[ "2017B" ] = [
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root"
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root"
]
fileNames_data[ "2017C" ] = [
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_0.root",
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_1.root"
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_1.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0001/SlimmedNtuple_merged_0.root"
]
fileNames_data[ "2017D" ] = [
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-Dilepton/190621_214154/SlimmedNtuple_merged_0.root"
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-withDilepton/0000/SlimmedNtuple_merged_1.root"
]
fileNames_data[ "2017E" ] = [
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_0.root",
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_1.root"
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/0000/SlimmedNtuple_merged_1.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-withDilepton/0001/SlimmedNtuple_merged_0.root"
]
fileNames_data[ "2017F" ] = [
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000/SlimmedNtuple_merged_0.root",
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000/SlimmedNtuple_merged_1.root"
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0001/SlimmedNtuple_merged_0.root"
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_1.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_1.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0002/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0003/SlimmedNtuple_merged_0.root"
]

label = "data-test"
#label = "data-random-resample_20-test"
firstEvent = None
entryStop = None
#read_size = "150MB"
step_size = 100000
random_protons = False
resample_factor = -1
#random_protons = True
#resample_factor = 20

if run_tables:
    for key_ in fileNames_data:
        print ( key_, fileNames_data[ key_ ] )
        label_ = "{}-{}".format( label, key_ )
        create_table( fileNames_data[ key_ ], label=label_, random_protons=random_protons, resample_factor=resample_factor, step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        

2017B ['/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root']
Random protons: False
Resample: False / Resample factor: -1
Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 74007
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_

2017C ['/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_0.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_1.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0001/SlimmedNtuple_merged_0.root']
Random protons: False
Resample: False / Resample factor: -1
Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 67415
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_

Slice: 1
Writing in positions (24762,60867)
9030 [{run: 300576, event: 525490238, lumiblock: 344, nVertices: 22, ... 116, 123, 123]}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ... 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1]
Num protons: [3, 3, 2, 1, 3, 4, 5, 1, 1, 3, 3, 5, 3, 3, ... 2, 2, 5, 3, 2, 2, 6, 1, 3, 2, 0, 2, 3]
Num pps tracks: [11, 3, 3, 1, 2, 12, 5, 1, 3, 2, 4, 4, 2, ... 4, 2, 5, 6, 4, 2, 9, 12, 3, 3, 0, 2, 7]
Num protons: [3, 3, 2, 1, 3, 4, 5, 1, 1, 3, 3, 5, 3, 3, ... 2, 2, 5, 3, 2, 2, 6, 1, 3, 2, 0, 2, 3]
Num pps tracks: [11, 3, 3, 1, 2, 12, 5, 1, 3, 2, 4, 4, 2, ... 4, 2, 5, 6, 4, 2, 9, 12, 3, 3, 0, 2, 7]
Num protons extra: [1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, ... 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

Num protons RP 3: [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]

Num protons RP 23: [0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 2, 1, 0

Slice: 1
Writing in positions (63891,95981)
Reduce dataset.
Dataset shape: (32090, 53)
Dataset shape: (71395, 49)
Dataset shape: (95981, 7)
Writing column names and event counts.
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0_mass' b'jet0_corrmass' b'jet0_tau1' b'jet0_tau2'
 b'jet0_vertexz' b'muon0_pt' b'muon0_eta' b'muon0_phi' b'muon0_energy'
 b'muon0_charge' b'muon0_iso' b'muon0_dxy' b'muon0_dz' b'met' b'met_x'
 b'met_y' b'met_phi' b'nVertices' b'num_bjets_ak8' b'num_bjets_ak4'
 b'num_jets_ak4' b'pfcand_nextracks' b'pfcand_nextracks_noDRl'
 b'recoMWhad' b'recoMWlep' b'recoMWW' b'recoRapidityWW' b'dphiWW'
 b'WLeptonicPt' b'WLeptonicPhi' b'trackx1' b'tracky1' b'trackpixshift1'
 b'rpid1' b'trackx2' b'tracky2' b'trackpixshift2' b'rpid2']
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0

Flattened array size multi-RP: 9164
Flattened array size single-RP: 21529
Flattened array size tracks: 24780
Stacking data.
(9164, 53)
[[ 3.02031000e+05  1.00000000e+00  5.92629000e+05 ... -3.57365608e-03
   0.00000000e+00  2.30000000e+01]
 [ 3.02031000e+05  1.00000000e+00  5.92629000e+05 ...  1.64965820e+00
   0.00000000e+00  1.23000000e+02]
 [ 3.02031000e+05  1.47000000e+02  1.58208892e+08 ... -1.26196289e+00
   0.00000000e+00  2.30000000e+01]
 ...
 [ 3.02485000e+05  2.00000000e+00  1.96934300e+06 ...  4.31445312e+00
   0.00000000e+00  1.23000000e+02]
 [ 3.02031000e+05  3.20000000e+02  3.42956994e+08 ...  2.57397461e+00
   0.00000000e+00  2.30000000e+01]
 [ 3.02031000e+05  3.20000000e+02  3.42956994e+08 ... -2.18399048e-01
   0.00000000e+00  1.23000000e+02]]
(21529, 49)
[[ 3.02031000e+05  1.00000000e+00  5.92629000e+05 ... -3.11832428e-02
   0.00000000e+00  3.00000000e+00]
 [ 3.02031000e+05  1.00000000e+00  5.92629000e+05 ... -3.57365608e-03
   0.00000000e+00  2.30000000e+01]
 [ 3.02

...

27551 [{run: 304562, event: 1307366462, lumiblock: 762, nVertices: 49, ... 23, 23, 23]}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, ... 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2]
Num protons: [0, 1, 4, 0, 0, 4, 3, 3, 3, 4, 2, 0, 3, 2, ... 4, 3, 2, 4, 3, 1, 3, 6, 0, 4, 5, 5, 3]
Num pps tracks: [0, 2, 5, 0, 0, 5, 2, 3, 4, 3, 3, 0, 3, 3, ... 5, 3, 2, 3, 3, 2, 2, 7, 0, 4, 5, 4, 4]
Num protons: [1, 4, 0, 0, 4, 3, 3, 3, 4, 2, 0, 3, 2, 5, ... 4, 3, 2, 4, 3, 1, 3, 6, 0, 4, 5, 5, 3]
Num pps tracks: [2, 5, 0, 0, 5, 2, 3, 4, 3, 3, 0, 3, 3, 6, ... 5, 3, 2, 3, 3, 2, 2, 7, 0, 4, 5, 4, 4]
Num protons extra: [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, ... 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0]

Num protons RP 3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Num protons RP 23: [1, 0, 0, 0, 0, 0, 3, 2, 1, 2, 0, 2, 0, 2, ... 1, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2, 2, 3]


2017F ['/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_0.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_1.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_0.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_1.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0002/SlimmedNtuple_merged_0.root', '/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0003/SlimmedNtuple_merged_0.root']
Random protons: False
Resample: False / Resample factor: -1
Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0000/SlimmedNtuple_merged_0.root
Number



4866
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ... 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
['All' 'Jet' 'Muon' 'Proton']
[147459 147459 145015   7880]
Flattened array size multi-RP: 9732
Flattened array size single-RP: 23187
Flattened array size tracks: 23789
Stacking data.
(9732, 53)
[[ 3.05821000e+05  3.61000000e+02  6.01379808e+08 ...  4.07196045e-01
   0.00000000e+00  2.30000000e+01]
 [ 3.05821000e+05  3.61000000e+02  6.01379808e+08 ...  2.28466797e+00
   0.00000000e+00  1.23000000e+02]
 [ 3.05862000e+05  6.33000000e+02  1.04293015e+09 ... -3.03881836e+00
   0.00000000e+00  2.30000000e+01]
 ...
 [ 3.05112000e+05  1.50500000e+03  2.43146885e+09 ...  2.11645508e+00
   0.00000000e+00  1.23000000e+02]
 [ 3.05377000e+05  6.41000000e+02  1.15488504e+09 ... -6.70996094e+00
   0.00000000e+00  2.30000000e+01]
 [ 3.05377000e+05  6.41000000e+02  1.15488504e+09 ...  2.41601562e+00
   0.00000000e+00  1.23000000e+02]]
(23187, 49)
[[ 3.05821000e+05  3.61000000e+02  6.01379808e+08 ...  3.1079101

60834 [{run: 305204, event: 1294702217, lumiblock: 860, ... pps_track_rpid: []}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num protons: [2, 5, 0, 0, 5, 0, 1, 1, 0, 1, 3, 2, 4, 4, ... 3, 3, 2, 3, 3, 4, 3, 5, 6, 1, 5, 7, 0]
Num pps tracks: [2, 4, 0, 0, 5, 15, 1, 1, 0, 6, 2, 2, 5, ... 5, 2, 3, 3, 2, 3, 4, 14, 7, 1, 4, 9, 0]
Num protons: [2, 5, 0, 0, 5, 0, 1, 1, 0, 1, 3, 2, 4, 4, ... 3, 3, 2, 3, 3, 4, 3, 5, 6, 1, 5, 7, 0]
Num pps tracks: [2, 4, 0, 0, 5, 15, 1, 1, 0, 6, 2, 2, 5, ... 5, 2, 3, 3, 2, 3, 4, 14, 7, 1, 4, 9, 0]
Num protons extra: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, ... 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0]

Num protons RP 3: [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, ... 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]

Num protons RP 23: [0, 2, 0, 0, 2, 0, 1, 0, 0, 1, 1, 1, 4, 2, ... 0, 0, 0, 1, 1, 2, 0, 4, 4, 1, 2, 1, 0]

Num proto

/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0003/SlimmedNtuple_merged_0.root
Number of events in tree: 82632
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', 'proton_trackx2', 'proton_tracky2', 'proton_trackpixshift1', 'proton_trackpixshift2', 'proton_

In [11]:
run_tables = True

debug = False

label = "data-random-resample_20-test"
firstEvent = None
entryStop = None
#read_size = "150MB"
step_size = 50000
random_protons = True
resample_factor = 20

if run_tables:
    for key_ in fileNames_data:
        print ( key_, fileNames_data[ key_ ] )
        label_ = "{}-{}".format( label, key_ )
        create_table( fileNames_data[ key_ ], label=label_, random_protons=random_protons, resample_factor=resample_factor, step_size=step_size, firstEvent=firstEvent, entryStop=entryStop, debug=debug )
        

2017B ['/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root']
Random protons: True
Resample: True / Resample factor: 20
Initial dataset shape: (50000, 53)
Initial dataset shape: (50000, 49)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-withDilepton/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 74007
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz

Flattened array size single-RP: 97360
Flattened array size tracks: 106020
Resizing output dataset by 50000 entries.
Dataset shape: (150000, 53)
Resizing output dataset by 100000 entries.
Dataset shape: (350000, 49)
Resizing output dataset by 100000 entries.
Dataset shape: (350000, 7)
Stacking data.
(45160, 53)
[[ 2.97056000e+05  7.00000000e+01  9.15369440e+07 ...  2.78106689e-01
   0.00000000e+00  2.30000000e+01]
 [ 2.97056000e+05  7.00000000e+01  9.15369440e+07 ... -7.20031738e-01
   0.00000000e+00  1.23000000e+02]
 [ 2.97056000e+05  7.00000000e+01  9.02331670e+07 ...  1.16479492e+00
   0.00000000e+00  2.30000000e+01]
 ...
 [ 2.97292000e+05  3.61000000e+02  6.48401492e+08 ... -4.37500000e-01
   0.00000000e+00  1.23000000e+02]
 [ 2.97292000e+05  3.83000000e+02  6.89092010e+08 ...  1.85424805e+00
   0.00000000e+00  2.30000000e+01]
 [ 2.97292000e+05  3.83000000e+02  6.89092010e+08 ...  2.79479980e-01
   0.00000000e+00  1.23000000e+02]]
(97360, 49)
[[ 2.97056000e+05  7.00000000e+01  9.153

50000 [{run: 299480, event: 317391607, lumiblock: 217, nVertices: 27, ... 103, 16, 23]}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num protons: [8, 1, 5, 2, 0, 3, 1, 4, 8, 1, 3, 4, 3, 4, ... 2, 3, 0, 8, 2, 3, 0, 0, 3, 2, 4, 0, 4]
Num pps tracks: [16, 1, 5, 4, 0, 3, 1, 5, 9, 1, 4, 6, 3, ... 4, 9, 0, 9, 3, 4, 2, 0, 4, 4, 5, 4, 4]
Num protons: [8, 1, 5, 2, 0, 3, 1, 4, 8, 1, 3, 4, 3, 4, ... 2, 3, 0, 8, 2, 3, 0, 0, 3, 2, 4, 0, 4]
Num protons randomized: [0, 6, 0, 6, 0, 3, 3, 0, 6, 6, 1, 7, 3, ... 2, 4, 3, 6, 0, 3, 2, 6, 10, 0, 2, 0, 1]
Num pps tracks: [16, 1, 5, 4, 0, 3, 1, 5, 9, 1, 4, 6, 3, ... 4, 9, 0, 9, 3, 4, 2, 0, 4, 4, 5, 4, 4]
Num pps tracks randomized: [0, 6, 1, 5, 13, 2, 2, 0, 6, 14, 1, 9, 3, ... 17, 2, 5, 0, 5, 4, 8, 10, 0, 3, 1, 2]
Num protons extra: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, ... 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,

Slice: 6
Writing in positions (255460,344280)
Slice: 9
Writing in positions (370440,495240)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-withDilepton/0000/SlimmedNtuple_merged_1.root
Number of events in tree: 109030
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', '

(364860, 7)
[[ 3.00155000e+05  4.39000000e+02  5.80082692e+08 ...  1.09887695e+01
   3.40795898e+00  3.00000000e+00]
 [ 3.00155000e+05  4.39000000e+02  5.80082692e+08 ...  5.28710938e+00
   4.10937500e+00  1.03000000e+02]
 [ 3.00155000e+05  4.39000000e+02  5.80082692e+08 ...  6.52569580e+00
   0.00000000e+00  1.60000000e+01]
 ...
 [ 3.00576000e+05  6.40000000e+01  9.30006940e+07 ...  4.69018555e+00
  -1.01796875e+01  2.30000000e+01]
 [ 3.00576000e+05  6.40000000e+01  9.30006940e+07 ...  5.37347412e+00
   1.06848145e+00  2.30000000e+01]
 [ 3.00576000e+05  6.40000000e+01  9.30006940e+07 ...  9.39343262e+00
   1.40502930e+00  1.23000000e+02]]
Slice: 7
Writing in positions (278440,397920)
Slice: 17
Writing in positions (602180,858000)
Slice: 24
Writing in positions (852480,1217340)
9030 [{run: 300576, event: 525490238, lumiblock: 344, nVertices: 22, ... 116, 123, 123]}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [1, 1, 1, 1, 1

...

Num protons: [3, 5, 6, 0, 12, 7, 4, 8, 3, 5, 3, 1, 5, ... 0, 1, 7, 4, 4, 9, 4, 3, 3, 3, 2, 7, 3]
Num protons randomized: [1, 7, 1, 6, 6, 1, 7, 0, 4, 5, 2, 8, 0, 1, ... 0, 5, 5, 3, 2, 3, 6, 4, 5, 0, 5, 5, 7]
Num pps tracks: [3, 6, 6, 0, 13, 8, 5, 9, 3, 4, 4, 1, 5, ... 0, 1, 5, 4, 3, 8, 3, 2, 2, 3, 2, 5, 2]
Num pps tracks randomized: [1, 7, 1, 5, 6, 1, 7, 0, 3, 4, 2, 9, 0, 1, ... 0, 5, 5, 2, 2, 2, 5, 4, 4, 0, 4, 4, 5]
Num protons extra: [0, 0, 1, 0, 2, 0, 0, 1, 0, 1, 0, 0, 1, 0, ... 0, 0, 2, 0, 1, 1, 1, 1, 1, 0, 0, 2, 1]
Num protons extra randomized: [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, ... 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 2]
Num protons: [1, 7, 1, 6, 6, 1, 7, 0, 4, 5, 2, 8, 0, 1, ... 0, 5, 5, 3, 2, 3, 6, 4, 5, 0, 5, 5, 7]
Num pps tracks: [1, 7, 1, 5, 6, 1, 7, 0, 3, 4, 2, 9, 0, 1, ... 0, 5, 5, 2, 2, 2, 5, 4, 4, 0, 4, 4, 5]
Num protons extra: [0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, ... 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 2]

Num protons RP 3: [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 

Slice: 20
Writing in positions (754840,1017700)
Slice: 20
Writing in positions (775860,1046200)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0001/SlimmedNtuple_merged_1.root
Number of events in tree: 60834
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1'

Slice: 27
Writing in positions (1296340,1357860)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0002/SlimmedNtuple_merged_0.root
Number of events in tree: 81586
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky1', 'proton_trackx2', 'proton_tracky2', 'proton_t

Slice: 34
Writing in positions (1574060,1740600)
Slice: 35
Writing in positions (1616660,1784140)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-withDilepton/0003/SlimmedNtuple_merged_0.root
Number of events in tree: 82632
['run', 'event', 'lumiblock', 'nVertices', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'met', 'met_x', 'met_y', 'met_phi', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'proton_time', 'proton_trackx1', 'proton_tracky

Slice: 42
Writing in positions (1961940,2102100)
Slice: 42
Writing in positions (2007880,2149480)
Reduce dataset.
Dataset shape: (849640, 53)
Dataset shape: (2102100, 49)
Dataset shape: (2149480, 7)
Writing column names and event counts.
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0_mass' b'jet0_corrmass' b'jet0_tau1' b'jet0_tau2'
 b'jet0_vertexz' b'muon0_pt' b'muon0_eta' b'muon0_phi' b'muon0_energy'
 b'muon0_charge' b'muon0_iso' b'muon0_dxy' b'muon0_dz' b'met' b'met_x'
 b'met_y' b'met_phi' b'nVertices' b'num_bjets_ak8' b'num_bjets_ak4'
 b'num_jets_ak4' b'pfcand_nextracks' b'pfcand_nextracks_noDRl'
 b'recoMWhad' b'recoMWlep' b'recoMWW' b'recoRapidityWW' b'dphiWW'
 b'WLeptonicPt' b'WLeptonicPhi' b'trackx1' b'tracky1' b'trackpixshift1'
 b'rpid1' b'trackx2' b'tracky2' b'trackpixshift2' b'rpid2']
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm

In [ ]:
def get_data( fileNames ):
    df_protons_multiRP_list = []
    df_protons_singleRP_list = []
    df_ppstracks_list = []
    df_counts_list = []

    for file_ in fileNames:
        print ( file_ )
        with h5py.File( file_, 'r' ) as f:
            print ( list(f.keys()) )

            dset_protons_multiRP = f['protons_multiRP']
            print ( dset_protons_multiRP.shape )
            print ( dset_protons_multiRP[:,:] )

            dset_protons_singleRP = f['protons_singleRP']
            print ( dset_protons_singleRP.shape )
            print ( dset_protons_singleRP[:,:] )

            dset_ppstracks = f['ppstracks']
            print ( dset_ppstracks.shape )
            print ( dset_ppstracks[:,:] )

            dset_columns_protons_multiRP = f['columns_protons_multiRP']
            print ( dset_columns_protons_multiRP.shape )
            columns_protons_multiRP = [ item.decode("utf-8") for item in list( dset_columns_protons_multiRP ) ]
            print ( columns_protons_multiRP )

            dset_columns_protons_singleRP = f['columns_protons_singleRP']
            print ( dset_columns_protons_singleRP.shape )
            columns_protons_singleRP = [ item.decode("utf-8") for item in list( dset_columns_protons_singleRP ) ]
            print ( columns_protons_singleRP )
            
            dset_columns_ppstracks = f['columns_ppstracks']
            print ( dset_columns_ppstracks.shape )
            columns_ppstracks = [ item.decode("utf-8") for item in list( dset_columns_ppstracks ) ]
            print ( columns_ppstracks )

            dset_selections = f['selections']
            selections_ = [ item.decode("utf-8") for item in dset_selections ]
            print ( selections_ )

            dset_counts = f['event_counts']
            df_counts_list.append( pd.Series( dset_counts, index=selections_ ) )
            print ( df_counts_list[-1] )

            chunk_size = 1000000
            entries_protons_multiRP = dset_protons_multiRP.shape[0]
            start_ = list( range( 0, entries_protons_multiRP, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries_protons_multiRP )
            print ( start_ )
            print ( stop_ )
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset_protons_multiRP[ start_[idx] : stop_[idx] ], columns=columns_protons_multiRP ).astype(
                                        { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
                                          "muon0_charge": "int32",
                                          "nVertices": "int32",
                                          "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
                                          "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32",
                                          "trackpixshift1": "int32", "rpid1": "int32",
                                          "trackpixshift2": "int32", "rpid2": "int32" } )
                df_protons_multiRP_list.append( df_ )
                print ( df_protons_multiRP_list[-1].head() )
                print ( "Data set size: {}".format( len( df_protons_multiRP_list[-1] ) ) )

            entries_protons_singleRP = dset_protons_singleRP.shape[0]
            start_ = list( range( 0, entries_protons_singleRP, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries_protons_singleRP )
            print ( start_ )
            print ( stop_ )
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset_protons_singleRP[ start_[idx] : stop_[idx] ], columns=columns_protons_singleRP ).astype(
                                        { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
                                          "muon0_charge": "int32",
                                          "nVertices": "int32",
                                          "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
                                          "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32",
                                          "trackpixshift1": "int32", "rpid1": "int32" } )
                df_protons_singleRP_list.append( df_ )
                print ( df_protons_singleRP_list[-1].head() )
                print ( "Data set size: {}".format( len( df_protons_singleRP_list[-1] ) ) )

            entries_ppstracks = dset_ppstracks.shape[0]
            start_ = list( range( 0, entries_ppstracks, chunk_size ) )
            stop_ = start_[1:]
            stop_.append( entries_ppstracks )
            print ( start_ )
            print ( stop_ )
            for idx in range( len( start_ ) ):
                print ( start_[idx], stop_[idx] )
                #print ( dset[ start_[idx] : stop_[idx] ] )
                df_ = pd.DataFrame( dset_ppstracks[ start_[idx] : stop_[idx] ], columns=columns_ppstracks ).astype( { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "rpid": "int32" } )
                df_ppstracks_list.append( df_ )
                print ( df_ppstracks_list[-1].head() )
                print ( "Data set size: {}".format( len( df_ppstracks_list[-1] ) ) )

    df_counts = df_counts_list[0]
    for idx in range( 1, len( df_counts_list ) ):
        df_counts = df_counts.add( df_counts_list[idx] )
    print ( df_counts )

    df_protons_multiRP = pd.concat( df_protons_multiRP_list )
    print (df_protons_multiRP)

    df_protons_singleRP = pd.concat( df_protons_singleRP_list )
    print (df_protons_singleRP)

    df_ppstracks = pd.concat( df_ppstracks_list )
    print (df_ppstracks)
    
    return ( df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks )
    

In [ ]:
fileNames_data = [
    'output-data-test-2017B.h5',
#     'output-data-test-2017C.h5',
#     'output-data-test-2017D.h5',
#     'output-data-test-2017E.h5',
#     'output-data-test-2017F.h5'
]

df_counts, df_protons_multiRP, df_protons_singleRP, df_ppstracks = get_data( fileNames_data )

In [ ]:
df_protons_multiRP[ ['run', 'lumiblock', 'event', 'slice'] ]

In [ ]:
df_protons_multiRP_index = df_protons_multiRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )
df_protons_multiRP_index

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP[ df_protons_multiRP.arm == 0 ].hist( "xi", bins=10, range=(0.,0.2), ax=axes[0] )
df_protons_multiRP[ df_protons_multiRP.arm == 1 ].hist( "xi", bins=10, range=(0.,0.2), ax=axes[1] )

In [ ]:
df_protons_multiRP_events = df_protons_multiRP_index.drop( columns=[ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm" ] )
df_protons_multiRP_events = df_protons_multiRP_events[ ~df_protons_multiRP_events.index.duplicated(keep='first') ]
df_protons_multiRP_events

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "recoMWW", bins=50, range=(200.,2000.), ax=axes[0] )
df_protons_multiRP_events.hist( "recoRapidityWW", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
#list( df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ) )
#df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: print( df_.iloc[0].xi, df_.iloc[1].xi ) ) 
df_protons_multiRP_events[ "MX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: 13000. * np.sqrt( df_.iloc[0].xi * df_.iloc[1].xi ) )
df_protons_multiRP_events[ "YX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event","slice"] ).apply( lambda df_: 0.5 * np.log( df_.iloc[0].xi / df_.iloc[1].xi ) )
df_protons_multiRP_events[ "diffMWW_MX" ]  = df_protons_multiRP_events[ "recoMWW" ] - df_protons_multiRP_events[ "MX" ]
df_protons_multiRP_events[ "ratioMWW_MX" ] = df_protons_multiRP_events[ "recoMWW" ] / df_protons_multiRP_events[ "MX" ]
df_protons_multiRP_events[ "diffYWW_YX" ]  = df_protons_multiRP_events[ "recoRapidityWW" ] - df_protons_multiRP_events[ "YX" ]
df_protons_multiRP_events

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "MX", bins=50, range=(200.,2000.), ax=axes[0] )
df_protons_multiRP_events.hist( "YX", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "diffMWW_MX", bins=50, range=(-1000.,1000.), ax=axes[0] )
df_protons_multiRP_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
df_protons_multiRP_events.hist( "ratioMWW_MX", bins=50, range=(0.,1.5), figsize=(12,10) )

In [ ]:
fig = plt.figure( figsize=(12,10) )
plt.plot( (-2000.,2000.), (0.,0.), color='lightblue' )
plt.plot( (0.,0.), (-3.,3.), color='lightblue' )
ax = plt.gca()
df_protons_multiRP_events.plot( "diffMWW_MX", "diffYWW_YX", 'scatter', ax=ax )
plt.xlim(-2000., 2000.)
plt.ylim(-3., 3.)

In [ ]:
df_protons_singleRP

In [ ]:
df_protons_singleRP_index = df_protons_singleRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )
df_protons_singleRP_index

In [ ]:
df_ppstracks

In [ ]:
df_ppstracks_index = df_ppstracks.set_index( ['run', 'lumiblock', 'event', 'slice'] )
#df_ppstracks_index
df_ppstracks_index[ ( df_ppstracks_index.rpid != 16 ) & ( df_ppstracks_index.rpid != 116 ) ]

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_ppstracks[ df_ppstracks.rpid == 23 ].hist( "x", bins=40, range=(0.,80.), ax=axes[0] )
df_ppstracks[ df_ppstracks.rpid == 23 ].hist( "y", bins=20, range=(-15.,15.), ax=axes[1] )

### References

In [ ]:
np.info( ak.concatenate )

In [ ]:
np.info( pd.DataFrame.hist )

In [ ]:
np.info( pd.DataFrame.plot.scatter )

In [ ]:
np.info( plt.plot )